In [1]:
import logging
import requests
import queue
from bs4 import BeautifulSoup
import threading
import json
from urllib.parse import urljoin

In [2]:
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:62.0) Gecko/20100101 Firefox/62.0'}
logging.basicConfig(level=logging.INFO)
cache_filename = 'cache.json'

In [ ]:
def get_bloges_link():
    with open('Resource/bookmarks_6_25_19.html') as bm:
        soup = BeautifulSoup(bm.read(), 'lxml')
        for item in soup.find_all('a'):
            if item['href'] is None:
                logging.warning("{} has no href".format(item))
                continue
            yield {'url': item['href'], 'name': item.string, 'added_time': item['add_date']}

In [ ]:
def gethtmldoc(url):
    if cache.get(url, None) is not None:
        logging.info('命中缓存')
        return cache[url]
    else:
        try:
            html = requests.get(url, allow_redirects=True, timeout=20, headers=headers)
        except BaseException:
            logging.error("Can't get {}.".format(url))
            return None
        if html.status_code is not 200:
            logging.error("HTTP STATUS {} in {}".format(url, html.status_code))
            return None
        cache[url] = html.text
        return html.text

In [ ]:
def worker():
    while True:
        item = q.get()
        if item is None:
            break
        htmldoc = gethtmldoc(item['url'])
        if htmldoc:
            soup = BeautifulSoup(htmldoc, 'lxml')
            fail = True
            for i in ['application/atom+xml', 'application/rss+xml', 'application/rdf+xml']:
                if soup.find(type=i) is not None:
                    print("Found {} in {}".format(soup.find(type=i)['href'], item['url']))
                    if soup.find(type=i)['href'][0:4] is 'http':
                        result[soup.find(type=i)['href']] = 1
                    else:
                        result[urljoin(item['url'], soup.find(type=i)['href'])] = 1
                    fail = False
                    break
            if fail:
                logging.error("Can't find RSS link in {}[{}]".format(item['url'], item['name']))
            else:
                pass
        else:
            pass
        q.task_done()

In [ ]:
def main():
    num_worker_threads = 64

    threads = []

    for i in range(num_worker_threads):
        t = threading.Thread(target=worker)
        t.start()
        threads.append(t)
    logging.info('线程已启动')
    for item in get_bloges_link():
        q.put(item)
    logging.info('目标网页已加载到队列')
    q.join()
    logging.info('等待任务完成...')
    for i in range(num_worker_threads):
        q.put(None)
    for t in threads:
        t.join()
    logging.info('再见')

In [ ]:
if __name__ == '__main__':
    result = {}
    q = queue.Queue()
    with open(cache_filename, 'r') as cache_f:
        cache = json.load(cache_f)
        logging.info('缓存已载入')
    main()

In [ ]:
with open(cache_filename, 'w') as f:
    json.dump(cache, f)

In [ ]:
for feed_url in result:
    try:
        r = requests.get(feed_url, allow_redirects=True, timeout=20)
    except BaseException:
        continue
    else:
        soup = BeautifulSoup(r.text, 'xml')
        for desc in soup.find_all('description'):
            print(desc.string)

我是小狐狸~欢迎来我的小窝！
<p>FoxWinShave 2 现已上架 GameMaker: Marketplace，支持 GM8, GMS 1 [&#8230;]</p>
<p><a rel="nofollow" href="https://www.noisyfox.io/foxwinshave2.html">FoxWinShave 2 中文使用手册</a>，首发于<a rel="nofollow" href="https://www.noisyfox.io">狐狸的小小窝</a>。</p>

<p>起因 前几天发现自己的博客没有办法登录了，每次输完密码登录完都会重新跳回登录界面。一开始是怀疑登录的sessi [&#8230;]</p>
<p><a rel="nofollow" href="https://www.noisyfox.io/logrotate.html">Logrotate 日志滚动 解决日志占用空间过大</a>，首发于<a rel="nofollow" href="https://www.noisyfox.io">狐狸的小小窝</a>。</p>

<p>Deprecated / 已弃用 In 2016, the experimental use in Chrom [&#8230;]</p>
<p><a rel="nofollow" href="https://www.noisyfox.io/cecpq1-nginx-boringssl.html">Use CECPQ1 On Your Website with Nginx &#038; BoringSSL</a>，首发于<a rel="nofollow" href="https://www.noisyfox.io">狐狸的小小窝</a>。</p>

<p>Unmanaged Exports 是一个将.NET编写的静态托管函数导出成可供C/C++等直接调用的非托管函 [&#8230;]</p>
<p><a rel="nofollow" href="https://www.noisyfox.io/unmanaged-exports.html">Unmanaged Exports 修正版 .NET导出非托管函数</a>，首发于<a rel="nofollow" href="https://www.noi

Thoughts, stories and ideas.
<p><img src="https://static.is26.com/blog/2019/06/newyork/nyc-143.JPG" alt="cover">
2018年上半年，有一天，无聊，在手机上刷海南航空，发现了上海往返纽约，不到人民币3000的往返机票，和老婆去了美国两次，都是走的美国西部自驾，还没去东部逛过。</p>

<ul>
<li><a href="https://luolei.org/usa-road-travel-nov/">美国 | 美西8000里:蜜月旅行，再续「前」缘</a></li>
<li><a href="https://luolei.org/usa-road-travel-feb/">美国 | 美西6000里:一趟旅行，女友成了前女友</a></li>
</ul>

<p>过去这几年，去了不少海岛，去了不少户外，见识了不少风光美景，乃至对海岛什么的都有点腻烦了，突然想找个城市看看，既然飞纽约的机票这么便宜，就干脆安排一次美东旅行吧，纽约、费城、华盛顿，也当做是一次人文行。就这样，买下了两张2019元旦假期，<strong>"上海&lt;>重庆&lt;>纽约"</strong>的往返机票。</p>

<p>计划赶不上变化，下半年，同老婆双双离职，离开了魔都，回到了深圳，开了自己的公司，开始了夫妻创业维艰。没了收入，原本的纽约、费城、华盛顿之行，</p>
<p><img src="https://static.is26.com/vlog/vlog41.jpg" alt="cover"></p>

<p>今天来跟大家分享一个比较小众的数码产品，目前我正在使用的手机卡还挺多的:</p>

<ul>
<li>中国电信:一张主卡，两张副卡，办理宽带送的，40G流量。</li>
<li>中国联通:用了N年的老号码，绑定了很多账号。</li>
<li>Cuniq香港:联通香港上台的号码</li>
<li>Lycamobile:美国号码（已翻车，不再提供中国漫游）</li>
</ul>

<p>平时出门，一般都只会带两个手机，一台iPhone XS，一台一加6，只能放三张卡，始终有几张卡，被迫插在家里的红米Note，收一收短信用。</p>

<p>由于Android限制，只有短信能够通

专注于前端开发

            <div class="toc"><ul>
<li><a href="#toc-319">SpriteJS 的特点</a></li>
<li><a href="#spritetexture">Sprite texture</a></li>
<li><a href="#toc-c3b">绘制矢量图</a></li>
<li><a href="#toc-829">分组</a></li>
<li><a href="#toc-d4b">响应事件</a></li>
<li><a href="#toc-e99">SpriteJS和D3</a></li>
<li><a href="#toc-f08">物理引擎</a></li>
<li><a href="#toc-386">粒子系统</a></li>
<li><a href="#toc-d65">外部时钟</a></li>
</ul>
</div><p>SpriteJS是一款由360奇舞团开源的跨终端canvas绘图库，可以基于canvas快速绘制结构化UI、动画和交互效果，并发布到任何拥有canvas环境的平台上（比如浏览器、小程序和node）。</p>
<p><a class="jsbin-embed" href="//code.h5jun.com/dexol/1/embed?output">JS Bin on jsbin.com</a><script src="https://code.h5jun.com/js/embed.min.js?3.40.2"></script></p>
<p>我们知道，Canvas Api可以很灵活地绘制各种矢量图形到画布上，但是Canvas Api本身比较低级，比如我们要在画布中央绘制一个带有圆角的红色矩形，使用Canvas原生的Api，需要这样：</p>
<p><a href="//code.h5jun.com/jobu/1/edit?js,output">JSBIN</a></p>
<pre><code class="hljs lang-js"><span class="hljs-keyword">const</span> canvas = <span class="hljs-built_in">document</span>.getElementB

和光同尘，与时舒卷
汝果欲学诗，工夫在诗外。 周末晚上去海边散步，听到沙滩上有音乐声。我走过去一看，原来有现场节目表演。 起先是来 [&#8230;]<div class="crp_related "><h3>Related Posts</h3><ul><li><a href="https://tumutanzi.com/archives/16616"    ><span class="crp_title">笑话与段子</span></a></li><li><a href="https://tumutanzi.com/archives/3206"    ><span class="crp_title">韩寒和李承鹏的两篇文章</span></a></li><li><a href="https://tumutanzi.com/archives/16595"    ><span class="crp_title">@steem-ua 代理收益分析：小额代理划算，但是…</span></a></li><li><a href="https://tumutanzi.com/archives/15269"    ><span class="crp_title">土木坛子微信公众号恢复更新</span></a></li></ul><div class="crp_clear"></div></div>
听说一个叫波场币创业者的孙晨宇拍得了巴菲特的午餐，一顿饭好几千万元人民币。 听说一个叫比特易的创始人玩100倍 [&#8230;]<div id="crp_related"> </div>
2010年春节前夕，我还在欧洲，惊闻Google“主动”退出中国大陆市场。从此至今，虽然几度传闻回归，Goog [&#8230;]<div class="crp_related "><h3>Related Posts</h3><ul><li><a href="https://tumutanzi.com/archives/6575"    ><span class="crp_title">那些攻而不破的谣言</span></a></li></ul><div class="crp_clear"></div></div>
最近几天，Bitcoin的价格终于回到6000美元以上了。半年前从一直横盘的630

Recent content in 开源的技术网站 on SuRenPi
本文介绍 Jenkins 相关的内容，包括：如何启动、配置等等。
部署 Jenkins 支持几种安装（启动）方式：
  War 由于 Jenkins 是用 Java 语言来开发的，自然就会有 War 的打包方式。你需要首先在下载页面，选择一个版本。
然后，执行命令 java -jar jenkins.war 启动后，就可以通过浏览器访问 http://localhost:8080/jenkins
这种方式，方便简单，但是需要有 Java 环境。
Docker 用容器的方式来运行 Jenkins 也很方便。只要执行命令 docker run -p 8080:8080 jenkins/jenkins:lts 即可。
需要每周更新版的话，直接加上版本号即可 docker run -p 12345:8080 jenkins/jenkins:2.175
挂载卷 docker run -v /your/home:/var/jenkins_home jenkins/jenkins:lts
Kubernetes Helm 执行命令 helm install stable/jenkins --name my-jenkins
如果希望看到具体内容请查看这里。
KubeApps 上也可以找到 Jenkins
版本选择 从 Jenkins 官网的下载页面，看到有很多版本可以下载，但是，具体该怎么选择呢。我下面给出一些建议：
 生产环境：最近一年内的 LTS（长期支持版） 相对正式的环境：最新的 LTS 体验：最新的 Weekly 插件研发：尽可能选择较老的 LTS  参数 Jenkins 在启动过程中，大多数的参数是具有默认值的。但是，对于一些特定的场景下，我们需要设置对应的参数值。
None
返回最新的一次提交记录 git log -1
查看某个文件的提交记录 git log somefile
查看某个文件的变更内容 git log -p Jenkinsfile
docker pull cptactionhank/atlassian-jira-software:latest
curl http://localhost:8080/user/admin/descriptorByName/jenkins.security

每一个可以努力的日子，都是一份厚礼。
由于 DNS 192.241.222.103 被广泛滥用，加上目前 DDoS 攻击十分猖獗，我们的服务器不堪重&#8230;
知乎上有一个提问，我们是如何一步一步走向平庸的？其中的 一个回答 让我印象深刻。 文中提到了一个英文单词 ma&#8230;
本文的读者需要已经了解 基本的 Git 操作和开发流程。 在我们开发完分支后，一般分支上会有很多 commit&#8230;
“最后一个问题，你为什么会加入这间公司？” 当这位前 Facebook 早期员工坐在我们办公室沙发上，突然问我&#8230;
老早就想写这个话题了，今天正好有机会研究了一下 git 的自动化部署。最终做到的效果就是，每当有新的 comm&#8230;
突然感到无比赞同 Livid 关于“信息量”的阐述。 信息不对等在这个世界无处不在。香港本地的电子商务并不发达&#8230;
Git 常用命令备忘 Git 配置 git config --global user.email "i@lov&#8230;
前天帮同事改个十年前的网站 bug，页面上一堆 include require 不禁让人抱头痛哭。看到 V2E&#8230;
公司内部原本使用 SVN 进行版本控制，但随着 Github 的流行我个人的代码管理习惯逐渐转变。虽然公司项目&#8230;
国内的优酷、土豆、搜狐、爱奇艺等网站上的很多正版影视资源屏蔽了海外 IP，仅限中国大陆用户观看。为了解除地域限&#8230;
None
<h2>02：狐狸</H2>
<p>猎人设下捕猎陷阱，一只狐狸不小心掉了进去，腿被夹住了，无法逃脱。</P>
<p>
猎人很是高兴，那可是只少见的红狐啊，光是那皮毛就能值不少钱。一想到钱，猎人心里就更高兴了，至少他儿子的婚事有着落了，现在穷人家找门合适的亲事可真不容易。</P>
<p>猎人来到陷阱旁，没想到那只狐狸却口吐人言：“求求您，放过我吧……”猎人断然拒绝：“我还想靠你去换我儿子的彩礼呢！”</P>
<p>“我上有高堂老母，下有新婚妻子，求您网开一面，放我一条生路吧……”狐狸苦苦哀求，“如果您放过我，日后我一定会报答您的。”</P>
<p>猎人听它这么一说，心里有点犹豫，但想到儿子的事情，还是一狠心，把狐狸打死了。</P>
<p>狐狸死后，悲愤难名：“人类如此铁石心肠！可怜我妻

None
<div NODE-TYPE="feed_list_content" NICK-NAME="非常真人"><a HREF="http://huati.weibo.com/k/HTC%E6%88%91%E4%BB%AC%E5%9C%A8%E4%B8%80%E8%B5%B7?from=501">#HTC我们在一起#</A>好朋友相聚，怎么能不拍张合影？HTC
Butterfly的广角镜头，能容纳更多人一起自拍，我们在一起，一个不能少！但这张照片却少了一个人，亲爱的你，快告诉我是谁吧？非常真人漫画第173集《一个不能少》<a HREF="http://weibo.com/n/HTC_Life">@HTC_Life</A> <a TITLE="http://e.weibo.com/htcwildfire/app_1395476762" HREF="http://t.cn/zYCaP8g" TARGET="_blank">http://t.cn/zYCaP8g</A></DIV>
<div NODE-TYPE="feed_list_content" NICK-NAME="非常真人">&nbsp;</DIV>
<div NODE-TYPE="feed_list_content" NICK-NAME="非常真人"><a href="http://blog.photo.sina.com.cn/showpic.html#url=http://s14.sinaimg.cn/orignal/49539201td8eabdfd16cd" TARGET="_blank"><img STYLE="TexT-ALiGn: center; DispLAY: block" SRC="http://s14.sinaimg.cn/mw690/49539201td8eabdfd16cd&amp;690" WIDTH="600" HEIGHT="2246" NAME="image_operate_83601364402644546" /></A><br />
<br /></DIV><br /><img src="http://simg.sinajs.cn/blog7style/images/special/1265.gif">&nbsp;
<div>
隐形的翅膀就是科技的力量，智者善用科技手段，不

Stay hungry, Stay foolish
IBM Cloud Forum今年在南京，一天的会议相比较之前的云栖大会，虽然会议也有穿插了重点客户的经验分享以及产品推广，但IBM大会上内容的深度和高度是高出不少的，听下来也是有些干货的，下面...
最近用了几天时间为公司项目集成了全文搜索引擎，项目初步目标是用于搜索框的即时提示。数据需要从MySQL中同步过来，因为数据不小，因此需要考虑初次同步后进行持续的增量同步。这里用到的开源服务就是E...
最近在项目上有一个简单的需求，在使用CKEditor编辑器时，点击最大化按钮，需要让最大化的窗口获得焦点，弹出退出提示，同时提供按键盘ESC键退出全屏编辑模式。这个操作模式有点类似于在Chrom...
这本书豆瓣评分6.7左右（详细介绍见这里），的确写的比较浅显，但作者确实表达出了他的观点，给出了他的见闻，提醒读者要注意到严峻的形式。这点来说做到了表达他想表达的。给我的感觉就是，下游老人离我们...
本篇为我的读书笔记部分，因此就不在这里着重介绍这本书了，关于原则一书的介绍可以参考一下这里。生活原则 第一章1.1 做一个超级现实的人a.梦想+现实+决心=成功的生活每个人都需要判断自己最珍视的...
网站上线前往往需要做一些压力测试，防止高并发场景时服务器挂掉。压力测试工具有不少，但是最简单的应该算是Apache附带的Apache Benchmark（下面简称AB）了，这里我就对AB的安装使...
我们在本地开发时，由于现在大部分网络接入都在内网环境中，没有公网ip，因此想要访问本地开发服务进行测试是比较难的，这一点在微信开发时尤其是个问题。因为微信开发是需要与微信的测试服务器接口进行交互...
入手MacBook对于某些开发来说是件很方便的事情，特别是对于常用于Linux Server环境的PHP代码开发。无论是编码格式，还是常用环境配置，MacBook的Unix基础有着与Linux相...
每次换一台新电脑，或者强迫症的重装了一次系统之后，就必须重新配置一下本地的开发环境。对于配置本地开发环境，从很早以来就有多种方案，包括：从PHP, MySQL等官网下载并手工安装配置使用Wind...
前言对于Linux服务器，默认以密码登陆，安全性比较差，很早前就有计划修改为使用密钥登陆，一直觉得比较麻烦搁置着，其实步骤并不难，

In [3]:
with open(cache_filename, 'r') as cache_f:
    cache = json.load(cache_f)

In [23]:
keywords = ''
for blog in webs[4:]:
    soup = BeautifulSoup(blog, 'lxml')
    tag_of_keywords = soup.find('meta', {'name': 'keywords'})
    if tag_of_keywords:
        keywords += (',' + tag_of_keywords['content'])

In [7]:
webs = list(cache.values())

In [8]:
len(webs)

397

In [12]:
soup = BeautifulSoup(webs[9], 'lxml')

In [13]:
soup.string

In [24]:
keywords

",关注前端开发、Node.js,typecho,php,blog,é¼\xa0å°¾è\x8d\x89_æ\x96°æµªå\x8d\x9aå®¢,é¼\xa0å°¾è\x8d\x89,å¨\x9få\xad\x90,æ\x88\x91ä»¬,æ\x84\x9fæ\x81©,è¿\x99æ\x98¯,è¿\x98æ\x98¯,ç»\x8fä¸\x96,é\x83½æ\x98¯,æ\x8a¤å£«,æ\x83\x85æ\x84\x9f,æ\x83\x85æ\x84\x9f,å¸\x8cæ\x9c\x9bå°\x8få\xad¦,å\x8e\x9fæ\x99\x93å¨\x9f,æ\x83\x85æ\x84\x9f,å¨\x9få\xad\x90,é\x99\x95è¥¿,å®\x8bä½\x93,å¸\x8cæ\x9c\x9bå°\x8få\xad¦,å\x8e\x9fæ\x99\x93å¨\x9f,æ\x9d\x82è°\x88,å¨\x9få\xad\x90,å\x8c\x97äº¬å\x9f\x8e,åº·åº·,é\x80\x9aä¿¡ä¸\x96ç\x95\x8c,ç§»å\x8a¨äº\x92è\x81\x94ç½\x91,æ\x9d\x82è°\x88,åº·åº·,ä¸\x9cå\x8d\x8eé\x97¨,é\x87\x8dç\x82¹ä¸\xadå\xad¦,å\xad¦æ\xa0¡,æ\x99®é\x80\x9aä¸\xadå\xad¦,æ\x9d\x82è°\x88,åº·åº·,ä¸\x9cå\x9f\x8eå\x8cº,é\x87\x8dç\x82¹ä¸\xadå\xad¦,å¨\x9få\xad\x90,ç¬¬ä¸\x80å¿\x97æ\x84¿,æ\x9d\x82è°\x88,å\x8c\x97äº¬,å°\x91å¹´ä¸\xadå\x9b½è¯´,åº·åº·å°\x8få\xad¦,æ¯\x95ä¸\x9aå\x85¸ç¤¼,å\x8f²å®¶å°\x8få\xad¦,æ\x83\x85æ\x84\x9f,C++,cppblog,服务器,算法,博客,C++ blog,C++博客,triihsia,Trii Hsia,情绪,凯立德,华阳,CID3681,CE66,DS,G12,通话录音,HTC,Desire S,情绪21℃,博客,

In [25]:
import wordcloud

In [26]:
wordcloud.WordCloud().generate(keywords)

In [27]:
words = keywords.split(',')

In [28]:
len(words)

831

In [39]:
words_frq = {}
for word in words:
    word = word.strip().lower()
    words_frq[word] = words_frq.get(word, 0) + 1

In [40]:
sorted(words_frq.items(), key=lambda kv: kv[1], reverse=True)

[('blog', 24),
 ('', 14),
 ('linux', 14),
 ('php', 13),
 ('hexo', 13),
 ('next', 9),
 ('typecho', 8),
 ('python', 8),
 ('博客', 7),
 ('web', 7),
 ('c++', 6),
 ('javascript', 6),
 ('生活', 6),
 ('æ\x9d\x82è°\x88', 5),
 ('java', 5),
 ('å¨\x9få\xad\x90', 4),
 ('æ\x83\x85æ\x84\x9f', 4),
 ('docker', 4),
 ('åº·åº·', 3),
 ('windows', 3),
 ('firefox', 3),
 ('互联网', 3),
 ('c', 3),
 ('computer', 3),
 ('machine learning', 3),
 ('android', 3),
 ('网络', 3),
 ('golang', 3),
 ('vim', 3),
 ('technology', 3),
 ('program', 3),
 ('å¸\x8cæ\x9c\x9bå°\x8få\xad¦', 2),
 ('å\x8e\x9fæ\x99\x93å¨\x9f', 2),
 ('é\x87\x8dç\x82¹ä¸\xadå\xad¦', 2),
 ('算法', 2),
 ('个人网站', 2),
 ('css', 2),
 ('html', 2),
 ('漏洞挖掘', 2),
 ('dukeyin', 2),
 ('尹德磊的博客', 2),
 ('记录生活', 2),
 ('技术交流', 2),
 ('诗词歌赋', 2),
 ('画作赏析', 2),
 ('wordpress主题', 2),
 ('wordpress微博主题', 2),
 ('wordpress极简主题', 2),
 ('dozer', 2),
 ('嵌入式', 2),
 ('计算机', 2),
 ('科技', 2),
 ('photography', 2),
 ('taresky', 2),
 ('动漫', 2),
 ('二次元', 2),
 ('显卡', 2),
 ('chrome', 2),
 ('google', 2),
